In [1]:
import collections
import math
import os
import random

import numpy as np
import tensorflow as tf
import  nltk

In [2]:
ppp = nltk.data.load('../Downloads/ppp.txt', encoding='utf8')
words_p = nltk.tokenize.word_tokenize(ppp)[121:]


alw = nltk.data.load('../Downloads/alw.txt', encoding='utf8')
words_a = nltk.tokenize.word_tokenize(alw)[122:]

In [3]:
words = words_a + words_p

In [4]:
vocabulary_size = 40000

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
del words_a
del words_p # Hint to reduce memory.

data_index = 0

In [5]:
word_embed = np.load('./final_embed.npy')


In [6]:
sentences = []
sent = []
indices = []
for index, word in enumerate(words):
    if word in ['?','.','!']:
        sent += [word_embed[dictionary[word]]]
        sentences += [sent]
        sent = []
    elif word in ['Chapter', 'CHAPTER'] and words[index+1] in ['1','I']:
        print(words[index: index+10])
        indices += [len(sentences)]
    else:
        sent += [word_embed[dictionary[word]]]

['CHAPTER', 'I', '.', 'Down', 'the', 'Rabbit-Hole', 'Alice', 'was', 'beginning', 'to']
['Chapter', '1', 'It', 'is', 'a', 'truth', 'universally', 'acknowledged', ',', 'that']


In [7]:
max_len = len(max(sentences, key=len))
parsed = map(lambda x: x + [word_embed[dictionary['UNK']] for i in range(len(x), max_len)], 
                sentences)

In [8]:
x = np.array(list(parsed))

labels = [[1, 0] for i in range(indices[1])] + [[0,1] for i in range(indices[1], len(sentences))]
y = np.array(labels)
#test = np.stack((x,y), axis=1)

In [62]:
word_embed.shape

(40000, 300)

In [53]:
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuf = x[shuffle_indices]
y_shuf = y[shuffle_indices]

In [54]:
sample_ind = -1*int(.1*len(y))
xt, xd = x_shuf[:sample_ind], x_shuf[sample_ind]
yt, yd = y_shuf[:sample_ind], y_shuf[sample_ind]

In [59]:
class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, embeddings,
        l2_reg_lambda=0.0):
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        
        l2_loss = tf.constant(0.0)
        
        
        self.expanded_embeddings = tf.expand_dims(embeddings, -1)

        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-mxpl-%s" % filter_size):
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=.1), name="W")
                b = tf.Variable(tf.constant(.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.expanded_embeddings,
                    W,
                    strides=[1,1,1,1],
                    padding="VALID",
                    name="conv"
                    )
                
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1,1,1,1],
                    padding='VALID',
                    name="pool"
                )
                pooled_outputs.append(pooled)
            
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(3, pooled_outputs)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
        
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)
        
        with tf.name_scope("output"):
            W = tf.Variable(
                    tf.truncated_normal([num_filters_total, num_classes], 
                                    stddev=.1), name="W")
            b = tf.Variable(tf.constant(.1, shape=[num_classes]), name="b")
            
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")
            
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(self.scores, self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss
        
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [60]:
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=True,
      log_device_placement=False)
    sess = tf.Session(config=session_conf)
    sav
    with sess.as_default():
        tf.Variable()
        cnn = TextCNN(
        sequence_length=xt.shape[1],
        num_classes=yt.shape[1],
        vocab_size=vocabulary_size,
        embedding_size=200,
        filter_sizes=[3,4,5],
        num_filters=128,
        embeddings=word_embed,
        l2_reg_lambda=0.0)
        
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.histogram_summary("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.scalar_summary("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.merge_summary(grad_summaries)
        
        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.scalar_summary("loss", cnn.loss)
        acc_summary = tf.scalar_summary("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.merge_summary([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.train.SummaryWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.merge_summary([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.train.SummaryWriter(dev_summary_dir, sess.graph)


        # Checkpointing
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        # Tensorflow assumes this directory already exists so we need to create it
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.all_variables())
        
        sess.run(tf.initialize_all_variables())
        
        
        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: .5
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)
            
            
        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = data_helpers.batch_iter(
            zip(x_train, y_train), 64, 200)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % 100 == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % 100 == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

ValueError: Shape must be rank 4 but is rank 3

In [49]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int(len(data)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

## The Plan

### Step 1
Train some actual word2vec sets. Understand how to use it. Then use CNNs for Language Understanding Stuff.
Use Gutenberg Library. Don't even think about changing the functions yet. Just use the for now.

### Step 2
Work through some analogical reasoning stuff. Practice optimizing, but ONLY as a means of understanding how to work with stuff. Try to understand how you would do author recognition with CNNs.

### Step 3
Read more papers. Understand more about what you've actually been doing. Write up some stuff on the link between CNNs and what you're doing. 

### Step 4
Implement a version of word2vec that allows for an increase in the vocabulary. 

### Step 5a
Implement a version of word2vec that adds bigram transitions. The main focus here is clustering transitions and integration of unknown vocab. When we see a known transition, we add it into the context.

### Step 5b
Add a graph on top of the network and begin looking at methods to choose link likelihood

### Step 6
Combine network and word2vec with bigram transitions. Recognize words as transitions. Especially that. Actually. You might even just try recognizing 3 word transitions like $a \xrightarrow{b} c$, that's probably the most concrete place to start. Also play around with a high dropout rate. So train batch. Drop / cluster a bunch then continue.

### Step 7
Try implimenting adding in trigrams inbetween bigrams. Basically, train two contexts once you've trained bigrams.
Also, implement adding of links for likely things. We are trying to optimize the likelihood, given some word, of the links from that word. 

### Step 7b
This is never going to work

### Step 8
Try to forget the hierarchical stuff. Try seeing how high features can get. Assume a fixed vocabulary size, but words not in the vocab can be added as higher level relations. Actually, try to merge transitions with words. Fixed graph size $|V|^2$

### Step 9a
Try to implement some fancy stability constraint stuff

### Step 9b
Try to get this system to recognize authors. You can train a classical neural network, but try it at different levels. Try to recognize authors using only the transitions between words. Even if this just performs decent, what does that mean? What interpretation could one draw. Perhaps this is closer to the idealized style recognition that has been hoped for in the past. Also, we can dry to reduce this to the minimum need transitions or maybe try to find singular identifying transitions per author.

### Step 10
Try to forcast the next sentence from the previous, or the next paragraph from the previous, etc. Try speech generation. Try to untangle mixed up words.